Install 
    numpy, 
    pandas, 
    seaborn, 
    plotly, 
    matplotlib, 
    scikit-learn, 
    scipy,
    collections

Import

Set warnings to ignore

In [40]:
import os

%pip install numpy --upgrade
%pip install pandas --upgrade
%pip install seaborn --upgrade
%pip install plotly --upgrade
%pip install matplotlib --upgrade
%pip install scikit-learn --upgrade
%pip install scipy --upgrade
%pip install collections --upgrade

import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from scipy.spatial.distance import cdist
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated pa

ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


Install
    Spotipy

Import 
    Spotipy oauth2

In [41]:
%pip install spotipy --upgrade

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



Read CSV data into variables

In [42]:
data = pd.read_csv("data\data.csv")
genre_data = pd.read_csv('data\data_by_genres.csv')
year_data = pd.read_csv('data\data_by_year.csv')

cluster_pipeline = Pipeline([('scaler', StandardScaler()), ('kmeans', KMeans(n_clusters=10))])
X = genre_data.select_dtypes(np.number)
cluster_pipeline.fit(X)
genre_data['cluster'] = cluster_pipeline.predict(X)

tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=1))])
genre_embedding = tsne_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
projection['genres'] = genre_data['genres']
projection['cluster'] = genre_data['cluster']

song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', KMeans(n_clusters=20, 
                                   verbose=False))
                                 ], verbose=False)

X = data.select_dtypes(np.number)
number_cols = list(X.columns)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)
data['cluster_label'] = song_cluster_labels

pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2))])
song_embedding = pca_pipeline.fit_transform(X)
projection = pd.DataFrame(columns=['x', 'y'], data=song_embedding)
projection['title'] = data['name']
projection['cluster'] = data['cluster_label']

In [43]:
SPOTIPY_CLIENT_ID = open(r"spotipyclientid.txt")
SPOTIPY_CLIENT_SECRET = open(r"spotipyclientsecret.txt")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

In [44]:
def find_song(name, artists, year):
    song_data = defaultdict()
    results = sp.search(q='track: {} artists: {} year: {}'.format(name,artists,year), limit=1, type='track')
    if results['tracks']['items'] == []:
        return None

    results = results['tracks']['items'][0]
    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]

    song_data['name'] = [name]
    song_data['year'] = [year]
    song_data['explicit'] = [int(results['explicit'])]
    song_data['duration_ms'] = [results['duration_ms']]
    song_data['popularity'] = [results['popularity']]

    for key, value in audio_features.items():
        song_data[key] = value

    return pd.DataFrame(song_data)

number_cols = ['valence', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo']


def get_song_data(song, spotify_data):
    
    try:
        song_data = spotify_data[(spotify_data['name'] == song['name']) & (spotify_data['artists'] == song['artists']) & (spotify_data['year'] == song['year'])].iloc[0]
        return song_data
    
    except IndexError:
        return find_song(song['name'], song['artists'], song['year'])
        

def get_mean_vector(song_list, spotify_data):
    
    song_vectors = []
    
    for song in song_list:
        song_data = get_song_data(song, spotify_data)
        if song_data is None:
            print('Warning: {} does not exist in Spotify or in database'.format(song['name']))
            continue
        song_vector = song_data[number_cols].values
        song_vectors.append(song_vector)  
    
    song_matrix = np.array(list(song_vectors))
    return np.mean(song_matrix, axis=0)


def flatten_dict_list(dict_list):
    
    flattened_dict = defaultdict()
    for key in dict_list[0].keys():
        flattened_dict[key] = []
    
    for dictionary in dict_list:
        for key, value in dictionary.items():
            flattened_dict[key].append(value)
            
    return flattened_dict


def recommend_songs( song_list, spotify_data, n_songs=10):
    
    metadata_cols = ['name', 'year', 'artists']
    song_dict = flatten_dict_list(song_list)
    
    song_center = get_mean_vector(song_list, spotify_data)
    scaler = song_cluster_pipeline.steps[0][1]
    scaled_data = scaler.transform(spotify_data[number_cols])
    scaled_song_center = scaler.transform(song_center.reshape(1, -1))
    distances = cdist(scaled_song_center, scaled_data, 'cosine')
    index = list(np.argsort(distances)[:, :n_songs][0])

    rec_songs = spotify_data.iloc[index]
    rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
    return rec_songs[metadata_cols].to_dict(orient='records')

In [45]:
[recommend_songs([{'name': 'lil jeep', 'artists': "['Lil Peep']", 'year':2020}],  data)]

[[{'name': 'Come and See Me (feat. Drake)',
   'year': 2016,
   'artists': "['PARTYNEXTDOOR', 'Drake']"},
  {'name': 'Teenage Fever', 'year': 2017, 'artists': "['Drake']"},
  {'name': 'Come and See Me (feat. Drake)',
   'year': 2016,
   'artists': "['PARTYNEXTDOOR', 'Drake']"},
  {'name': "Don't", 'year': 2015, 'artists': "['Bryson Tiller']"},
  {'name': 'Money In The Grave (Drake ft. Rick Ross)',
   'year': 2019,
   'artists': "['Drake', 'Rick Ross']"},
  {'name': 'Greet Her', 'year': 2018, 'artists': "['Lil Mosey']"},
  {'name': 'Numb - Bonus Track',
   'year': 2014,
   'artists': "['August Alsina', 'B.o.B', 'Yo Gotti']"},
  {'name': 'Worthy', 'year': 2015, 'artists': "['Jeremih', 'Jhené Aiko']"},
  {'name': 'Hope You Do', 'year': 2017, 'artists': "['Chris Brown']"}]]